In [ ]:
import cv2
import numpy as np
import os
data_path = r"NOT_REAL_DATA\\"
# Load images using OpenCV
image_list = []
for filename in os.listdir(data_path):
    if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.tif'):
        img = cv2.imread(os.path.join(data_path, filename))
        image_list.append(img)

In [2]:

def crop_coin(image):  # Function now expects an OpenCV image directly
    """Crops the image to include only the coin and a little bit of black around it.

    Args:
        image (numpy.ndarray): The OpenCV image.

    Returns:
        numpy.ndarray: The cropped image.
    """

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to isolate the coin
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

    # Find contours in the thresholded image
    _, contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour that is most likely the coin
    coin_contour = None
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Check if the contour is circular and has a reasonable size
        if cv2.contourArea(contour) > 100 and w / h >= 0.8 and w / h <= 1.2:
            coin_contour = contour
            break

    # If a coin contour was found, crop the image
    if coin_contour is not None:
        x, y, w, h = cv2.boundingRect(coin_contour)
        # Add a small margin to the bounding box
        margin = 10
        cropped_image = image[y - margin:y + h + margin, x - margin:x + w + margin]
    else:
        print("Coin not found in image: Unable to determine filename") 
        cropped_image = None

    return cropped_image


# # Replace 'path/to/image.jpg' with the path to your image
# cropped_image = crop_coin('Image sent')

# # Display the cropped image
# if cropped_image is not None:
#     cv2.imshow("Cropped image", cropped_image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

In [ ]:
# # Process images
# cropped_images = []
# for img in image_list:
#     cropped_image = crop_coin(img)  # Pass the image directly
#     cropped_images.append(cropped_image)

# Process the first image separately
cropped = crop_coin(image_list[0])